In [13]:
from search_engine_versions.bm25_abstract_and_title import SearchEngine as bm25_abstract_and_title_search_engine
# ...

from query_evaluator import evaluate_articles_boolean, evaluate_articles_levels
from query_generator import generate_and_save_queries
import numpy as np
import pandas as pd
import json


# For queries gen
# path = './app/data/example.json'
# with open(path, 'r', encoding='utf-8') as f:
#     data = f.readlines()
#     data = [json.loads(line) for line in data]
# generate_and_save_queries(data, num_queries=100, output_path='./app/data/queries.json')

DATA_DIR = "./data/"
DATA_FILES = ["example.json"]
SPLIT_SIZE_LIMIT = 100

search_engines = {}
search_engines_dict = {
    "bm25_title_and_abstract": bm25_abstract_and_title_search_engine,
}

In [14]:
def average_precision(relevances):
    relevant = np.where(relevances == 1)[0] + 1  # get positions of relevant docs, +1 because index is zero-based
    if len(relevant) == 0:
        return 0
    precision_at_k = np.cumsum(relevances) / (np.arange(len(relevances)) + 1)
    return np.sum(precision_at_k[relevances == 1]) / len(relevant)

def dcg(relevances):
    k = len(relevances)
    relevances = np.asfarray(relevances)[:k]
    if relevances.size:
        return relevances[0] + np.sum(relevances[1:] / np.log2(np.arange(2, relevances.size + 1)))
    return 0

def ndcg(relevances):
    k = len(relevances)
    dcg_val = dcg(relevances)
    ideal_relevances = sorted(relevances, reverse=True)
    idcg_val = dcg(ideal_relevances)
    return dcg_val / idcg_val if idcg_val > 0 else 0

with open(DATA_DIR + "queries.json") as f:
    queries = f.readlines()
    queries = [json.loads(line) for line in queries]


def test_search_engine(SearchEngine, name):
    search_engine = SearchEngine()
    search_engine.feed_json(DATA_DIR, DATA_FILES, SPLIT_SIZE_LIMIT)
    search_engines[name] = search_engine

    aps = {}
    ndcgs_levels = {}
    ndcgs_boolean = {}

    for query_obj in queries[:10]:
        query = query_obj['query']
        search_results = search_engine.search(query, 5)
        search_results.rename(columns={'body': 'abstract'}, inplace=True)

        evaluated_results_boolean = evaluate_articles_boolean(search_results.to_dict(orient="records"), query, verbose=False)
        evaluated_results_boolean = pd.DataFrame(evaluated_results_boolean)
        evaluated_results_boolean['eval'] = pd.to_numeric(evaluated_results_boolean['eval'])
        print(evaluated_results_boolean)

        evaluated_results_levels = evaluate_articles_levels(search_results.to_dict(orient="records"), query, verbose=False)
        evaluated_results_levels = pd.DataFrame(evaluated_results_levels)
        evaluated_results_levels['eval'] = pd.to_numeric(evaluated_results_levels['eval'])
        print(evaluated_results_levels)

        aps[query_obj['id']] = average_precision(evaluated_results_boolean['eval'])
        ndcgs_levels[query_obj['id']] = ndcg(evaluated_results_levels['eval'])
        ndcgs_boolean[query_obj['id']] = ndcg(evaluated_results_boolean['eval'])
    
    metrics = {
        'aps': aps,
        'ndcgs_levels': ndcgs_levels,
        'ndcgs_boolean': ndcgs_boolean
    }
    return metrics


In [15]:
metrics_dict = {}
idx = 0
for name, search_engine in search_engines_dict.items():
    if idx > 3:
        break
    print('running for ', name)
    metrics = test_search_engine(search_engine, name)
    metrics_dict[name] = metrics
    idx+=1
    !docker stop $(docker ps -a -q) 

running for  bm25_title_and_abstract
Waiting for configuration server, 0/300 seconds...
Waiting for configuration server, 5/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 0/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 5/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 10/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Waiting for application status, 15/300 seconds...
Using plain http against endpoint http://localhost:8080/ApplicationStatus
Application is up!
Finished deployment.


Evaluating articles:  20%|██        | 1/5 [00:03<00:12,  3.09s/it]

id: 0704.0001


Evaluating articles:  40%|████      | 2/5 [00:03<00:05,  1.77s/it]

id: 0704.0060


...

Evaluating articles:   0%|          | 0/5 [00:00<?, ?it/s]

Error when parsing feedback: Invalid \escape: line 1 column 266 (char 265)
Error when parsing feedback: Expecting value: line 1 column 1 (char 0)


Evaluating articles: 100%|██████████| 5/5 [00:37<00:00,  7.49s/it]

                                               title  \
0  Partial cubes: structures, characterizations, ...   
1           Sparsity-certifying Graph Decompositions   
2  Quantum Field Theory on Curved Backgrounds. II...   
3  Approximate solutions to the Dirichlet problem...   
4       Conformal Field Theory and Operator Algebras   

                                            abstract  eval  
0    Partial cubes are isometric subgraphs of hyp...     3  
1    We describe a new algorithm, the $(k,\ell)$-...     1  
2    We study space-time symmetries in scalar qua...     0  
3    Our main result in this paper is the followi...     1  
4    We review recent progress in operator algebr...     0  


76f19d6a576c
abdc641f54d4
c870ed76a2df
454c00247a31
4b414421b8a8
458574697be5
421df1fdaee9
a28a62472faf
91cf3800e4ba
cc19e3046262
bce785b59fb3


In [16]:
metrics_dict

{'bm25_title_and_abstract': {'aps': {'0704.0001': 0.8333333333333333,
   '0704.0002': 0.8333333333333333,
   '0704.0003': 1.0,
   '0704.0004': 1.0,
   '0704.0005': 1.0,
   '0704.0006': 1.0,
   '0704.0007': 0.75,
   '0704.0008': 1.0,
   '0704.0009': 1.0,
   '0704.0010': 1.0},
  'ndcgs_levels': {'0704.0001': 1.0,
   '0704.0002': 1.0,
   '0704.0003': 1.0,
   '0704.0004': 0.9567574534371579,
   '0704.0005': 0.9609713628687518,
   '0704.0006': 1.0,
   '0704.0007': 0.9237991466948189,
   '0704.0008': 1.0,
   '0704.0009': 1.0,
   '0704.0010': 0.9717271130121371},
  'ndcgs_boolean': {'0704.0001': 0.8154648767857288,
   '0704.0002': 0.8154648767857288,
   '0704.0003': 1.0,
   '0704.0004': 1.0,
   '0704.0005': 1.0,
   '0704.0006': 1.0,
   '0704.0007': 0.75,
   '0704.0008': 1.0,
   '0704.0009': 1.0,
   '0704.0010': 1.0}}}

In [ ]:
from sheets_connection import SaveMetrics

save_metrics = SaveMetrics()

save_metrics.append_metrics('ndcg_binary_map', df.T)